In [1]:
import os
import sys

sys.dont_write_bytecode = True
os.environ["NUMBA_DISABLE_JIT"] = "1"
from quantfreedom.levon_qf.strategy_maker_cls_levon import StrategyMaker

import numpy as np
import pandas as pd

from quantfreedom.data.data_dl import data_download_from_ccxt
from quantfreedom.base import backtest_df_only
from quantfreedom.nb.simulate import simulate_up_to_6_nb
from quantfreedom.evaluators.evaluators import is_below
from quantfreedom.indicators.talib_ind import from_talib
from quantfreedom.enums.enums import (
    LeverageMode,
    SizeType,
    OrderType,
    CandleBody,
)
from quantfreedom.plotting.strat_dashboard import strat_dashboard

np.set_printoptions(formatter={"float_kind": "{:.2f}".format})
pd.options.display.float_format = "{:,.2f}".format

# prices = data_download_from_ccxt(
#     exchange='bybit',
#     start='2020-10-01T00:00:00Z',
#     end='2021-04-15T00:00:00Z',
#     symbols=['BTCUSDT', 'ETHUSD'],
#     timeframe='30m',
# )
sm = StrategyMaker()
# prices.to_hdf("yt_prices.hd5", key='data')
price_data = pd.read_hdf('../tests/data/prices.hd5')
rsi_ind = sm.from_talib(
    func_name="rsi",
    price_data=price_data,
    cart_product=False,
    combos=False,
    timeperiod=[15, 30],
)
entries = rsi_ind.is_below(
    user_args=[40, 50],
    cand_ohlc="close",
)
sm.combined_data_frame()

symbol              BTCUSDT                      EOSUSD                       \
RSI_timeperiod           15            30            15            30          
RSI_is_below             40     50     40     50     40     50     40     50   
open_time                                                                      
2020-10-01 00:00:00   False  False  False  False  False  False  False  False   
2020-10-01 00:30:00   False  False  False  False  False  False  False  False   
2020-10-01 01:00:00   False  False  False  False  False  False  False  False   
2020-10-01 01:30:00   False  False  False  False  False  False  False  False   
2020-10-01 02:00:00   False  False  False  False  False  False  False  False   
...                     ...    ...    ...    ...    ...    ...    ...    ...   
2021-04-14 21:30:00   False   True  False   True  False  False  False  False   
2021-04-14 22:00:00   False   True  False   True  False  False  False  False   
2021-04-14 22:30:00   False   True  False   True  False  False  False  False   
2021-04-14 23:00:00   False   True  False   True  False  False  False  False   
2021-04-14 23:30:00   False   True  False   True  False  False  False  False   

symbol              ETHUSD                      LTCUSDT                       \
RSI_timeperiod          15            30             15            30          
RSI_is_below            40     50     40     50      40     50     40     50   
open_time                                                                      
2020-10-01 00:00:00  False  False  False  False   False  False  False  False   
2020-10-01 00:30:00  False  False  False  False   False  False  False  False   
2020-10-01 01:00:00  False  False  False  False   False  False  False  False   
2020-10-01 01:30:00  False  False  False  False   False  False  False  False   
2020-10-01 02:00:00  False  False  False  False   False  False  False  False   
...                    ...    ...    ...    ...     ...    ...    ...    ...   
2021-04-14 21:30:00  False  False  False  False   False  False  False  False   
2021-04-14 22:00:00  False  False  False  False   False  False  False  False   
2021-04-14 22:30:00  False  False  False  False   False  False  False  False   
2021-04-14 23:00:00  False  False  False  False   False  False  False  False   
2021-04-14 23:30:00  False  False  False  False   False  False  False  False   

symbol              XRPUSD                       
RSI_timeperiod          15            30         
RSI_is_below            40     50     40     50  
open_time                                        
2020-10-01 00:00:00  False  False  False  False  
2020-10-01 00:30:00  False  False  False  False  
2020-10-01 01:00:00  False  False  False  False  
2020-10-01 01:30:00  False  False  False  False  
2020-10-01 02:00:00  False  False  False  False  
...                    ...    ...    ...    ...  
2021-04-14 21:30:00  False  False  False  False  
2021-04-14 22:00:00  False  False  False  False  
2021-04-14 22:30:00  False  False  False  False  
2021-04-14 23:00:00  False  False  False  False  
2021-04-14 23:30:00  False  False  False  False  

[9408 rows x 20 columns]

In [2]:
strat_results_df, settings_results_df = sm.backtest_df(
    prices=price_data,
    equity=1000.0,
    fee_pct=0.06,
    mmr_pct=0.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.LongEntry,
    max_equity_risk_pct=np.arange(7, 8.1, 1),
    risk_rewards=4,
    size_pct=1.0,
    # sl_pcts=np.arange(1,3.1,1),
    tsl_true_or_false=True,
    tsl_pcts_init=8,
    tsl_based_on=CandleBody.low,
    tsl_trail_by_pct=8,
    tsl_when_pct_from_avg_entry=np.arange(3, 4.1, 1),
    gains_pct_filter=150,
    upside_filter=0.6,
    divide_records_array_size_by=2,
)

Checking static variables for errors or conflicts.
Turning all variables into arrays.
Checking arrays for errors or conflicts ... the backtest will begin shortly, please hold.
Creating cartesian product ... after this the backtest will start, I promise :).

Starting the backtest now ... and also here are some stats for your backtest.

Total symbols: 5
Total indicator settings per symbol: 4
Total indicator settings to test: 20
Total order settings per symbol: 4
Total order settings to test: 20
Total candles per symbol: 9,408
Total candles to test: 752,640

Total combinations to test: 80


In [10]:
settings_results_df

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
symbol,BTCUSDT,BTCUSDT,BTCUSDT,BTCUSDT,BTCUSDT,BTCUSDT,BTCUSDT,BTCUSDT,BTCUSDT,BTCUSDT,...,ETHUSD,ETHUSD,ETHUSD,ETHUSD,ETHUSD,ETHUSD,ETHUSD,ETHUSD,LTCUSDT,LTCUSDT
entries_col,0,0,0,0,1,1,1,1,3,3,...,10,10,10,10,11,11,11,11,15,15
max_equity_risk_pct,7.00,7.00,8.00,8.00,7.00,7.00,8.00,8.00,7.00,7.00,...,7.00,7.00,8.00,8.00,7.00,7.00,8.00,8.00,7.00,8.00
risk_rewards,4.00,4.00,4.00,4.00,4.00,4.00,4.00,4.00,4.00,4.00,...,4.00,4.00,4.00,4.00,4.00,4.00,4.00,4.00,4.00,4.00
size_pct,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
tsl_based_on,low,low,low,low,low,low,low,low,low,low,...,low,low,low,low,low,low,low,low,low,low
tsl_pcts_init,8.00,8.00,8.00,8.00,8.00,8.00,8.00,8.00,8.00,8.00,...,8.00,8.00,8.00,8.00,8.00,8.00,8.00,8.00,8.00,8.00
tsl_trail_by_pct,8.00,8.00,8.00,8.00,8.00,8.00,8.00,8.00,8.00,8.00,...,8.00,8.00,8.00,8.00,8.00,8.00,8.00,8.00,8.00,8.00
tsl_when_pct_from_avg_entry,3.00,4.00,3.00,4.00,3.00,4.00,3.00,4.00,3.00,4.00,...,3.00,4.00,3.00,4.00,3.00,4.00,3.00,4.00,3.00,3.00


In [7]:
order_records = sm.select_row_and_simulate([1,22])

In [28]:
sm.indicators[0].data

symbol              BTCUSDT       EOSUSD       ETHUSD       LTCUSDT        \
RSI_timeperiod           15    30     15    30     15    30      15    30   
open_time                                                                   
2020-10-01 00:00:00     NaN   NaN    NaN   NaN    NaN   NaN     NaN   NaN   
2020-10-01 00:30:00     NaN   NaN    NaN   NaN    NaN   NaN     NaN   NaN   
2020-10-01 01:00:00     NaN   NaN    NaN   NaN    NaN   NaN     NaN   NaN   
2020-10-01 01:30:00     NaN   NaN    NaN   NaN    NaN   NaN     NaN   NaN   
2020-10-01 02:00:00     NaN   NaN    NaN   NaN    NaN   NaN     NaN   NaN   
...                     ...   ...    ...   ...    ...   ...     ...   ...   
2021-04-14 21:30:00   45.28 47.78  53.37 52.99  66.12 62.18   63.93 58.31   
2021-04-14 22:00:00   43.88 46.99  54.99 53.83  64.17 61.16   60.58 56.59   
2021-04-14 22:30:00   46.50 48.29  56.95 54.84  64.85 61.55   59.17 55.86   
2021-04-14 23:00:00   48.76 49.41  59.35 56.10  67.11 62.89   60.71 56.76   
2021-04-14 23:30:00   47.64 48.84  54.74 53.91  64.75 61.72   61.07 56.97   

symbol              XRPUSD        
RSI_timeperiod          15    30  
open_time                         
2020-10-01 00:00:00    NaN   NaN  
2020-10-01 00:30:00    NaN   NaN  
2020-10-01 01:00:00    NaN   NaN  
2020-10-01 01:30:00    NaN   NaN  
2020-10-01 02:00:00    NaN   NaN  
...                    ...   ...  
2021-04-14 21:30:00  56.21 53.93  
2021-04-14 22:00:00  52.11 51.85  
2021-04-14 22:30:00  53.82 52.75  
2021-04-14 23:00:00  56.84 54.34  
2021-04-14 23:30:00  57.71 54.81  

[9408 rows x 10 columns]

In [29]:
entries.iloc[:,[0]]

symbol,BTCUSDT
RSI_timeperiod,15
RSI_is_below,40
open_time,
2020-10-01 00:00:00,False
2020-10-01 00:30:00,False
2020-10-01 01:00:00,False
2020-10-01 01:30:00,False
2020-10-01 02:00:00,False
...,...
2021-04-14 21:30:00,False


In [17]:
indicator_dict = {
    "indicator1": {
        "values1": rsi_ind.data[[('BTCUSDT', 15)]],
        "entries1": entries.iloc[:,[0]],
    },
}
strat_dashboard(
    indicator_dict=indicator_dict,
    prices=price_data.BTCUSDT,
    order_records=order_records,
)

Dash is running on http://127.0.0.1:3003/

Dash app running on http://127.0.0.1:3003/


In [ ]:
indicator_dict = {
    "indicator1": {
        "values1": rsi_ind.data[[('BTCUSDT', 15)]],
        "values2": rsi_ema_ind.data[[('BTCUSDT', 15)]],
        "entries1": entries.iloc[:,[0]],
    },
    "indicator2": {
        "values1": macd.data[[('BTCUSDT', 15)]],
        "entries1": entries.iloc[:,[0]],
    },
}
strat_dashboard(
    indicator_dict=indicator_dict,
    prices=price_data.BTCUSDT,
    order_records=order_records,
)

Dash is running on http://127.0.0.1:3003/

Dash app running on http://127.0.0.1:3003/
